In [2]:
import ivy

In [5]:
import sys
sys.path.append("/workspaces/models_Ivy_Sark42/log_sys")
from pf import pf


In [17]:

class ConvBlock(ivy.Module):
    """
    Conv block used in the GoogLeNet architecture.

    Args::
        in_channels (int): Number of input channels.
        out_channels (int): Number of output channels.
        kernel_size ([list | tuple]): kernel_shape for the block.
        stride (Optional[ivy.Module]): Stride value for the block.

    """

    def __init__(self, in_channels, out_channels, kernel_size, stride, padding):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        super(ConvBlock, self).__init__()

    def _build(self, *args, **kwargs):
        self.conv = ivy.Conv2D(
            self.in_channels,
            self.out_channels,
            self.kernel_size,
            self.stride,
            self.padding,
            with_bias=False,
            data_format="NCHW",
        )
        self.bn = ivy.BatchNorm2D(self.out_channels, eps=0.001, data_format="NCS")
        self.relu = ivy.ReLU()

    def _forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        return x


def test_ConvBlock():
    ivy.set_backend('torch')
    # conv1
    # N x 224 x 224 x 3
    # random_test_tensor = ivy.random_normal(shape=(1, 224, 224, 3))
    random_test_tensor = ivy.random_normal(shape=(1, 3, 224, 224))

    pf(f"test_ConvBlock | random_test_tensor shape is: {random_test_tensor.shape}")

    block = ConvBlock(3, 64, [7,7], (2,2), 3)
    block(random_test_tensor)
    # N x 112 x 112 x 64
    pf("test_ConvBlock | Test Successfull!")
    pf("||")

test_ConvBlock()


In [23]:

class ConvBlock(ivy.Module):
    """
    Conv block used in the GoogLeNet architecture.

    Args::
        in_channels (int): Number of input channels.
        out_channels (int): Number of output channels.
        kernel_size ([list | tuple]): kernel_shape for the block.
        stride (Optional[ivy.Module]): Stride value for the block.

    """

    def __init__(self, in_channels, out_channels, kernel_size, stride, padding):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        super(ConvBlock, self).__init__()

    def _build(self, *args, **kwargs):
        self.conv = ivy.Conv2D(
            self.in_channels,
            self.out_channels,
            self.kernel_size,
            self.stride,
            self.padding,
            with_bias=False,
            data_format="NHWC",
        )
        self.bn = ivy.BatchNorm2D(self.out_channels, eps=0.001)
        self.relu = ivy.ReLU()

    def _forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        return x


def test_ConvBlock():
    ivy.set_backend('torch')
    # conv1
    # N x 224 x 224 x 3
    random_test_tensor = ivy.random_normal(shape=(1, 224, 224, 3))
    # random_test_tensor = ivy.random_normal(shape=(1, 3, 224, 224))

    pf(f"test_ConvBlock | random_test_tensor shape is: {random_test_tensor.shape}")

    block = ConvBlock(3, 64, [7,7], (2,2), 3)
    block(random_test_tensor)
    # N x 112 x 112 x 64
    pf("test_ConvBlock | Test Successfull!")
    pf("||")

test_ConvBlock()


In [28]:
class Inception(ivy.Module):
    """
    Inception block used in the GoogLeNet architecture.

    Args::
        in_channels (int): Number of input channels.
        num1x1 (int): Number of num1x1 channels.
        num3x3_reduce (int): Number of num3x3_reduce channels.
        num3x3 (int): Number of num3x3 channels.
        num5x5_reduce (int): Number of num5x5_reduce channels.
        num5x5 (int): Number of num5x5 channels.
        pool_proj (int): Number of pool_proj channels.
    """

    def __init__(
        self,
        in_channels,
        num1x1,
        num3x3_reduce,
        num3x3,
        num5x5_reduce,
        num5x5,
        pool_proj,
    ):
        super(Inception, self).__init__()
        self.in_channels = in_channels
        self.num1x1 = num1x1
        self.num3x3_reduce = num3x3_reduce
        self.num3x3 = num3x3
        self.num5x5_reduce = num5x5_reduce
        self.num5x5 = num5x5
        self.pool_proj = pool_proj,

    def _build(self, *args, **kwargs):
        self.conv_1x1 = ivy.Sequential(
            ConvBlock(
                self.in_channels,
                self.num1x1,
                kernel_size=[1, 1],
                stride=1,
                padding="SAME",
            )
        )

        self.conv_3x3 = ivy.Sequential(
            ConvBlock(
                self.in_channels,
                self.num3x3_reduce,
                kernel_size=[1, 1],
                stride=1,
                padding="SAME",
            ),
            ConvBlock(
                self.num3x3_reduce,
                self.num3x3,
                kernel_size=[3, 3],
                stride=1,
                padding="SAME",
            ),
        )

        self.conv_5x5 = ivy.Sequential(
            ConvBlock(
                self.in_channels,
                self.num5x5_reduce,
                kernel_size=[1, 1],
                stride=1,
                padding="SAME",
            ),
            ConvBlock(
                self.num5x5_reduce,
                self.num5x5,
                kernel_size=[3, 3],
                stride=1,
                padding="SAME",
            ),
        )

        self.pool_proj = ivy.Sequential(
            ivy.MaxPool2D([3, 3], 1, "SAME"),
            ConvBlock(
                self.in_channels,
                self.pool_proj,
                kernel_size=[1, 1],
                stride=1,
                padding="SAME",
            ),
        )

    def _forward(self, x):
        conv_1x1 = self.conv_1x1(x)

        conv_3x3 = self.conv_3x3(x)

        conv_5x5 = self.conv_5x5(x)

        pool_proj = self.pool_proj(x)

        return ivy.concat([conv_1x1, conv_3x3, conv_5x5, pool_proj], axis=3)


def test_Inception():
    ivy.set_backend('torch')
    # inception3a
    # N x 192 x 28 x 28
    random_test_tensor = ivy.random_normal(shape=(1, 28, 28, 192))
    # random_test_tensor = ivy.random_normal(shape=(1, 192, 28, 28))

    pf(f"test_Inception | random_test_tensor shape is: {random_test_tensor.shape}")

    block = Inception(192, 64, 96, 128, 16, 32, 32)
    block(random_test_tensor)
    # N x 256 x 28 x 28
    pf("test_Inception | Test Successfull!")
    pf("||")

test_Inception()



TypeError: unsupported operand type(s) for +: 'int' and 'tuple'

In [24]:

class Inception(ivy.Module):
    """
    Inception block used in the GoogLeNet architecture.

    Args::
        in_channels (int): Number of input channels.
        num1x1 (int): Number of num1x1 channels.
        num3x3_reduce (int): Number of num3x3_reduce channels.
        num3x3 (int): Number of num3x3 channels.
        num5x5_reduce (int): Number of num5x5_reduce channels.
        num5x5 (int): Number of num5x5 channels.
        pool_proj (int): Number of pool_proj channels.
    """

    def __init__(
        self,
        in_channels,
        num1x1,
        num3x3_reduce,
        num3x3,
        num5x5_reduce,
        num5x5,
        pool_proj,
    ):
        self.in_channels = in_channels
        self.num1x1 = num1x1
        self.num3x3_reduce = num3x3_reduce
        self.num3x3 = num3x3
        self.num5x5_reduce = num5x5_reduce
        self.num5x5 = num5x5
        self.pool_proj = pool_proj,
        super(Inception, self).__init__()

    def _build(self, *args, **kwargs):
        self.conv_1x1 = ivy.Sequential(
            ConvBlock(
                self.in_channels,
                self.num1x1,
                kernel_size=[1, 1],
                stride=1,
                padding="SAME",
            )
        )

        self.conv_3x3 = ivy.Sequential(
            ConvBlock(
                self.in_channels,
                self.num3x3_reduce,
                kernel_size=[1, 1],
                stride=1,
                padding="SAME",
            ),
            ConvBlock(
                self.num3x3_reduce,
                self.num3x3,
                kernel_size=[3, 3],
                stride=1,
                padding="SAME",
            ),
        )

        self.conv_5x5 = ivy.Sequential(
            ConvBlock(
                self.in_channels,
                self.num5x5_reduce,
                kernel_size=[1, 1],
                stride=1,
                padding="SAME",
            ),
            ConvBlock(
                self.num5x5_reduce,
                self.num5x5,
                kernel_size=[3, 3],
                stride=1,
                padding="SAME",
            ),
        )

        self.pool_proj = ivy.Sequential(
            ivy.MaxPool2D([3, 3], 1, "SAME", data_format="NCHW"),
            ConvBlock(
                self.in_channels,
                self.pool_proj,
                kernel_size=[1, 1],
                stride=1,
                padding="SAME",
            ),
        )

    def _forward(self, x):
        conv_1x1 = self.conv_1x1(x)

        conv_3x3 = self.conv_3x3(x)

        conv_5x5 = self.conv_5x5(x)

        pool_proj = self.pool_proj(x)

        return ivy.concat([conv_1x1, conv_3x3, conv_5x5, pool_proj], axis=3)


def test_Inception():
    ivy.set_backend('torch')
    # inception3a
    # N x 192 x 28 x 28
    # random_test_tensor = ivy.random_normal(shape=(1, 28, 28, 192))
    random_test_tensor = ivy.random_normal(shape=(1, 192, 28, 28))

    pf(f"test_Inception | random_test_tensor shape is: {random_test_tensor.shape}")

    block = Inception(192, 64, 96, 128, 16, 32, 32)
    block(random_test_tensor)
    # N x 256 x 28 x 28
    pf("test_Inception | Test Successfull!")
    pf("||")

test_Inception()



TypeError: unsupported operand type(s) for +: 'int' and 'tuple'